In [11]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time

def process(df):
#     print(df)
    return df

#一共19个类别，102500行
reader = pd.read_csv('/Volumes/KNIGHT/article_class/new_data/train_set.csv',sep=',',iterator=True) 
loop = True 
chunkSize = 100   # 这个chunk大小可以改 
full_result = pd.DataFrame([])
i= 1 
while i < 2: 
    i+=1
    start_time = time.time() 
    try: 
        chunk = reader.get_chunk(chunkSize) 
#         output = process(chunk)     # 这个process function 可以是任意的从dataframe到dataframe的function 
#         full_result = pd.concat([full_result,output]) 
    except StopIteration: 
        loop = False 
        print("Iteration is stopped.")
        print('i=',i)
# print(full_result[full_result['class']>=19])

Iteration is stopped.
i= 1025


In [56]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
t1=time.time()
train = pd.read_csv('../input/train_set.csv')
test = pd.read_csv('../input/test_set.csv')
test_id = pd.read_csv('../input/test_set.csv')[["id"]].copy()

column="word_seg"
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
trn_term_doc = vec.fit_transform(train[column])
test_term_doc = vec.transform(test[column])

y=(train["classify"]-1).astype(int)
clf = LogisticRegression(C=4, dual=True)
clf.fit(trn_term_doc, y)
preds=clf.predict_proba(test_term_doc)

#保存概率文件
test_prob=pd.DataFrame(preds)
test_prob.columns=["class_prob_%s"%i for i in range(1,preds.shape[1]+1)]
test_prob["id"]=list(test_id["id"])
test_prob.to_csv('../sub_prob/prob_lr_baseline.csv',index=None)

#生成提交结果
preds=np.argmax(preds,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv('../sub/sub_lr_baseline.csv',index=None)
t2=time.time()
print("time use:",t2-t1)

FileNotFoundError: File b'../input/train_set.csv' does not exist

In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time

def process(df,size):
    t1=time.time()
    train = pd.read_csv('/Volumes/KNIGHT/article_class/new_data/train_set.csv',sep=',',iterator=True).get_chunk(size)
    test = pd.read_csv('/Volumes/KNIGHT/article_class/new_data/test_set.csv',sep=',',iterator=True).get_chunk(size)
    test_id = pd.read_csv('/Volumes/KNIGHT/article_class/new_data/test_set.csv',sep=',',iterator=True).get_chunk(size)[["id"]].copy()

    column="word_seg"
    n = train.shape[0]
    vec = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
    trn_term_doc = vec.fit_transform(train[column])
    test_term_doc = vec.transform(test[column])

    y=(train["class"]-1).astype(int)
    clf = LogisticRegression(C=4, dual=True)
    clf.fit(trn_term_doc, y)
    preds=clf.predict_proba(test_term_doc)

    #保存概率文件
    test_prob=pd.DataFrame(preds)
    test_prob.columns=["class_prob_%s"%i for i in range(1,preds.shape[1]+1)]
    test_prob["id"]=list(test_id["id"])
    test_prob.to_csv('../article_classification/prob_lr_baseline.csv',mode='a',index=None)

    #生成提交结果
    preds=np.argmax(preds,axis=1)
    test_pred=pd.DataFrame(preds)
    test_pred.columns=["class"]
    test_pred["class"]=(test_pred["class"]+1).astype(int)
    print(test_pred.shape)
    print(test_id.shape)
    test_pred["id"]=list(test_id["id"])
    test_pred[["id","class"]].to_csv('../article_classification/sub_lr_baseline.csv',mode='a',index=None)
    t2=time.time()
    print("time use:",t2-t1)


#一共19个类别,102500行
reader = pd.read_csv('/Volumes/KNIGHT/article_class/new_data/train_set.csv',sep=',',iterator=True) 
loop = True 
chunkSize = 300   # 这个chunk大小可以改 
full_result = pd.DataFrame([])
i= 1 
while i < 3: 
    i+=1
    start_time = time.time() 
    try: 
        chunk = reader.get_chunk(chunkSize) 
        output = process(chunk,chunkSize)     # 这个process function 可以是任意的从dataframe到dataframe的function  
    except StopIteration: 
        loop = False 
        print("Iteration is stopped.")

(300, 1)
(300, 1)
time use: 2.5537469387054443
(300, 1)
(300, 1)
time use: 1.878183126449585


In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time

# 102500 * 19

def process(size,batch=3):
    head = 0
    loop = True 
    train_reader = pd.read_csv('/Volumes/KNIGHT/article_class/new_data/train_set.csv',sep=',',iterator=True)
    test_reader = pd.read_csv('/Volumes/KNIGHT/article_class/new_data/test_set.csv',sep=',',iterator=True)
    test_id_reader = pd.read_csv('/Volumes/KNIGHT/article_class/new_data/test_set.csv',sep=',',iterator=True)
    chunkSize = size   # 这个chunk大小可以改 
    i = 1
    while loop:
        i+=1
        try:
            t1=time.time()
            train = train_reader.get_chunk(chunkSize)
            test = test_reader.get_chunk(chunkSize)
            test_id = test_id_reader.get_chunk(chunkSize)[["id"]].copy()
        
            column="word_seg"
            n = train.shape[0]
            vec = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
            trn_term_doc = vec.fit_transform(train[column])
            test_term_doc = vec.transform(test[column])

            y=(train["class"]-1).astype(int)
            clf = LogisticRegression(C=4, dual=True)
            clf.fit(trn_term_doc, y)
            preds=clf.predict_proba(test_term_doc)

            #保存概率文件
            test_prob=pd.DataFrame(preds)
#             test_prob.columns=["class_prob_%s"%i for i in range(1,preds.shape[1]+1)]
#             test_prob["id"]=list(test_id["id"])
#             test_prob.to_csv('../article_classification/prob_lr_baseline.csv',mode='a',index=None)

            #生成提交结果
            preds=np.argmax(preds,axis=1)
            test_pred=pd.DataFrame(preds)
            test_pred.columns=["class"]
            test_pred["class"]=(test_pred["class"]+1).astype(int)
#             print(test_pred.shape)
#             print(test_id.shape)
            test_pred["id"]=list(test_id["id"])
            if head ==0:
                test_pred[["id","class"]].to_csv('../article_classification/sub_lr_baseline.csv',mode='a',index=None)
                head = 1
            elif head ==1:
                test_pred[["id","class"]].to_csv('../article_classification/sub_lr_baseline.csv',mode='a',index=None,header=False)
            t2=time.time()
            print('batch No.',i-1,", time use:",t2-t1)
        except StopIteration: 
            loop = False 
            print("Iteration is stopped.")
            
process(size=5000)


batch No. 2 , time use: 40.37032890319824
batch No. 3 , time use: 46.650410890579224
batch No. 4 , time use: 42.98404502868652
batch No. 5 , time use: 48.207704067230225
batch No. 6 , time use: 44.374927043914795
batch No. 7 , time use: 39.300575971603394
batch No. 8 , time use: 39.12427306175232
batch No. 9 , time use: 41.71040081977844
batch No. 10 , time use: 44.056488037109375
batch No. 11 , time use: 43.507827043533325
batch No. 12 , time use: 40.85246801376343
batch No. 13 , time use: 39.938934087753296
batch No. 14 , time use: 42.726608991622925
batch No. 15 , time use: 37.88478994369507
batch No. 16 , time use: 37.19625401496887
batch No. 17 , time use: 36.560173988342285
batch No. 18 , time use: 42.49337339401245
batch No. 19 , time use: 39.97270393371582
batch No. 20 , time use: 38.99448204040527
batch No. 21 , time use: 38.48114204406738
batch No. 22 , time use: 16.268331050872803
Iteration is stopped.
